In [4]:
from find_source import summary, significant
from astropy.io import fits
from astropy.coordinates import Angle, SkyCoord
import astropy.units as u
import glob

In [5]:
def get_info_for_catalog(fits_file: str):
    '''
    Summarizes information on any significant point sources detected in an image.

    Parameters
    ----------
    fits_file : str
        The path of the FITS file that contains the image.

    Returns
    -------
    dict
        A dictionary with:
            str
                The name of the target object of the observation.
            str
                The name of the FITS file with the image.
            float
                The uncertainty in flux density measurements.
            dict(s)
                A dictionary with:
                    float
                        The flux density of the detected point source.
                    SkyCoord
                        The location of the detected point source.
                    bool
                        Whether the detected point source is in the initial search region.
    '''

    summ = summary(fits_file, True, False, False)

    header_data = fits.getheader(fits_file)
    name = header_data['OBJECT']
    obs_date_time = header_data['DATE-OBS']
    bmaj = header_data['BMAJ']
    bmin = header_data['BMIN']
    bpa = header_data['BPA']
    ctype1 = header_data['CTYPE1']
    crval1 = header_data['CRVAL1']
    cunit1 = header_data['CUNIT1']
    ctype2 = header_data['CTYPE2']
    crval2 = header_data['CRVAL2']
    cunit2 = header_data['CUNIT2']

    #assume beam axes in same units as CUNIT1 and CUNIT2 and BPA in degrees
    beam_maj_axis = Angle(bmaj, cunit1) 
    beam_min_axis = Angle(bmin, cunit1) 
    beam_pos_angle = Angle(bpa, u.degree)
    
    interesting_sources = {'name': name, 'obs_date_time': obs_date_time, 'file': fits_file[fits_file.rindex('/')+1:],\
                           'beam_maj_axis': beam_maj_axis, 'beam_min_axis': beam_min_axis, 'beam_pos_angle': beam_pos_angle,\
                           'flux_uncertainty': summ['rms']} 

    n_ext_sources = 0
    if type(summ['ext_peak_val']) == list:
        n_ext_sources += len(summ['ext_peak_val'])

    ra_index = 0
    dec_index = 1

    if 'RA' in ctype1:
        ra = crval1
    elif 'RA' in ctype2:
        ra = crval2
        ra_index = 1
    else:
        raise ValueError('No RA in image')

    if 'DEC' in ctype1:
        dec = crval1
        dec_index = 0
    elif 'DEC' in ctype2:
        dec = crval2
    else:
        raise ValueError('No dec in image')

    if cunit1 != cunit2:
        raise ValueError('Axes have different units')

    center = SkyCoord(ra, dec, unit=cunit1)

    pt_source_count = 1

    if significant(fits_file):
        int_info = {}
        int_info['flux_density'] = summ['int_peak_val']

        int_ra_offset = summ['int_peak_coord'][ra_index] * u.arcsec
        int_dec_offset = summ['int_peak_coord'][dec_index] * u.arcsec
        int_info['coord'] = center.spherical_offsets_by(int_ra_offset, int_dec_offset)

        int_info['internal'] = True

        interesting_sources[f'source_{pt_source_count}'] = int_info
        pt_source_count +=1


    for i in range(n_ext_sources):
        ext_info = {}
        ext_info['flux_density'] = summ['ext_peak_val'][i]

        ext_ra_offset = summ['ext_peak_coord'][i][ra_index] * u.arcsec
        ext_dec_offset = summ['ext_peak_coord'][i][dec_index] * u.arcsec
        ext_info['coord'] = center.spherical_offsets_by(ext_ra_offset, ext_dec_offset)

        ext_info['internal'] = False

        interesting_sources[f'source_{pt_source_count}'] = ext_info
        pt_source_count += 1

    if 'source_1' not in interesting_sources:
        return
    else:
        return interesting_sources

In [6]:
for file in glob.glob('../data/multi_track/*.fits'):
    try:
        info = get_info_for_catalog(file)
        if info != None:
            print(info)
    except:
        print(f'Try again for {file}')

{'name': 'uranus', 'obs_date_time': '4-7-25 10:58:12', 'file': 'uranus_11128.fits', 'beam_maj_axis': <Angle 0.00117344 deg>, 'beam_min_axis': <Angle 0.0008631 deg>, 'beam_pos_angle': <Angle 107.9587 deg>, 'flux_uncertainty': 0.675508785998217, 'source_1': {'flux_density': 23.69101333618164, 'coord': <SkyCoord (ICRS): (ra, dec) in deg
    (38.57268009, 14.70263075)>, 'internal': True}}
{'name': '3c84', 'obs_date_time': '4-7-25 11:1:32', 'file': '3c84_11002.fits', 'beam_maj_axis': <Angle 0.00098876 deg>, 'beam_min_axis': <Angle 0.00072359 deg>, 'beam_pos_angle': <Angle 51.55273 deg>, 'flux_uncertainty': 0.01683113774577099, 'source_1': {'flux_density': 9.299407958984375, 'coord': <SkyCoord (ICRS): (ra, dec) in deg
    (49.95044418, 41.5115295)>, 'internal': True}, 'source_2': {'flux_density': 0.09259068965911865, 'coord': <SkyCoord (ICRS): (ra, dec) in deg
    (49.96179131, 41.51202879)>, 'internal': False}, 'source_3': {'flux_density': 0.09188197553157806, 'coord': <SkyCoord (ICRS): (ra